## Candidate Generation

Generate pairwise candidates from entities tagged in documents loaded above.

Note: This is often a good time to run the scripts/tagging-frequency.ipynb notebook before continuing to ensure that there are not problematic tags

In [1]:
from snorkel import SnorkelSession
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence, Candidate
from tcre import supervision
from tcre.query import DocToCand
from tcre.env import *
import numpy as np
import pandas as pd
import tqdm
clear_cands = False
max_sents = 100000000
classes = supervision.get_candidate_classes()

In [2]:
session = SnorkelSession()

In [3]:
session.query(Document).count()

64274

In [4]:
all_sent_ids = [r[0] for r in session.query(Sentence.id).all()]
len(all_sent_ids)

10996405

In [5]:
if clear_cands:
    print('Clearing existing candidates')
    session.query(Candidate).delete()
    session.commit()

In [6]:
if not clear_cands:
    print('Filtering sentences to those with no candidates')
    df = DocToCand.all(session, classes)
    sent_ids = list(set(all_sent_ids) - set(df['sentence_id'].unique()))
else:
    sent_ids = all_sent_ids
len(sent_ids)

Filtering sentences to those with no candidates


10915498

In [7]:
# Subsample to ensure that processing is tractable
if max_sents is not None and len(sent_ids) > max_sents:
    sent_ids = pd.Series(sent_ids).sample(max_sents, random_state=TCRE_SEED).tolist()
len(sent_ids)

10915498

In [8]:
from snorkel.candidates import PretaggedCandidateExtractor
candidate_extractors = [
    PretaggedCandidateExtractor(c.subclass, c.entity_types)
    for c in classes.values()
]

In [9]:
def apply_extraction(sent_ids, split, batch_size=50000):
    for extractor in candidate_extractors:
        relation_class = extractor.udf_init_kwargs['candidate_class']
        n_batch = int(np.ceil(len(sent_ids) / batch_size))
        print('Beginning candidate extraction for split {}, relation type {}, num batches {}'.format(
            split, relation_class.__name__, n_batch
        ))
        for ids in tqdm.tqdm(np.array_split(list(sent_ids), n_batch)):
            ids = [int(i) for i in ids]
            sents = session.query(Sentence).filter(Sentence.id.in_(frozenset(ids))).all()
            assert len(sents) == len(ids), f'Sents len = {len(sents)}, ids len = {len(ids)}'
            extractor.apply(sents, split=split, clear=False, progress_bar=False)
        print('Number of candidates generated for split {}, relation type {} = {}'.format(
            split, relation_class.__name__,
            session.query(relation_class).filter(relation_class.split == split).count()
        ))

apply_extraction(sent_ids, supervision.SPLIT_INFER)

Beginning candidate extraction for split 9, relation type InducingCytokine, num batches 219


  0%|          | 0/219 [00:00<?, ?it/s]

Running UDF...


  0%|          | 1/219 [00:17<1:03:49, 17.57s/it]

Running UDF...


...

Running UDF...


100%|█████████▉| 218/219 [1:08:26<00:17, 17.73s/it]

Running UDF...


100%|██████████| 219/219 [1:08:46<00:00, 18.38s/it]


Number of candidates generated for split 9, relation type InducingTranscriptionFactor = 158977


In [10]:
cands = session.query(Candidate.type, Candidate.split).all()
pd.DataFrame(cands).groupby(['type', 'split']).size()

type                           split
inducing_cytokine              0         10000
                               1          1025
                               2           371
                               3           278
                               9        566192
inducing_transcription_factor  0         10000
                               1           627
                               2           300
                               3           225
                               9        158977
secreted_cytokine              0         10000
                               1          1019
                               2           364
                               3           272
                               9        566211
dtype: int64